In [1]:
import openai
import os




In [2]:
# 定義 prompt
prompt_def = """
Please define the following categories:
1. Celebrity
2. Politician
3. Journalist/Media
4. General Public
5. Perpetrators
6. Victims
"""
response_def = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt_def}
    ]
)

print(response_def.choices[0].message["content"])


1. Celebrity: A celebrity refers to a person who has gained public recognition and popularity, typically through their work in areas such as entertainment, sports or other media-oriented fields. These individuals often use their fame to influence trends, culture, and public opinion.

2. Politician: A politician is an individual who is involved in influencing or making public policy decisions, usually by holding an elected position in a government office. This can include roles from local governance like mayors or council members, to national roles like senators, representatives, or even the president or prime minister.

3. Journalist/Media: Journalists or media personnel work in broadcasting, print, or digital platforms, providing news and information to the public. Their job often involves researching, reporting, writing, presenting, or editing news stories. The term "media" refers to entities that provide information, such as newspapers, television, radio, websites and social media p

In [3]:
# 假設 GPT 回答了某個定義，然後你在第二步的 prompt 中包含之前的對話
conversation_history = f"User: {prompt_def}\nAssistant: {response_def.choices[0].message["content"]}"

# 第二步請求，基於上一步的對話
promp_2 = """
Here are some individuals. For each, categorize them as either "celebrity", "politician", "journalist/media", "general public", "perpetrators" or "victims":

1. Barack Obama
2. Taylor Swift
3. Anderson Cooper
4. A random person on the street
5. A guy killed a woman
6. A woman got murdered 

Provide the classification in the following format:
- Name: [Classification]
"""
prompt_test = f"{conversation_history}\nUser: {promp_2}"
response_test = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt_test}
    ]
)
print(response_test.choices[0].message["content"])


1. Barack Obama: Politician
2. Taylor Swift: Celebrity
3. Anderson Cooper: Journalist/Media
4. A random person on the street: General Public
5. A guy killed a woman: Perpetrator
6. A woman got murdered: Victim


In [4]:
def classify_identities(article_text):
    prompt = f"""
    Analyze the following text and extract the names and their identities as either "celebrity", "politician", "journalist/media", "perpetrator", "victim" or "general public" by definition provided. 

    Definition:
    {conversation_history}
    
    Text:
    {article_text}
    Also, you should determine whether the entity is "asian", "non-asian" or "unknown".
    
    Provide the result in the following JSON format: 
    {{
    "entities": [
        {{
            "name": "",
            "identity": ""
            "isasian": ""
        }},
        {{
            "name": "",
            "identity": ""
            "isasian": ""
        }}
    ]
    }}
    After completing the analysis, provide your JSON output within <result> tags
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message["content"]




In [5]:
valid_identities = ["celebrity", "politician", "journalist/media", "general public", "perpetrator", "victim"]
valid_isasian = ["asian", "non-asian", "unknown"]
# 檢查 JSON 格式和 identity 是否有效
def validate_json_and_identity(json_data):
    try:        
        # 檢查是否存在 "entities" 字段，並且它是否是一個列表
        if "entities" in json_data and isinstance(json_data["entities"], list):
            # 檢查每個實體是否有 "name" 和 "identity" 字段，並且 identity 是否有效
            for entity in json_data["entities"]:
                if "name" not in entity or "identity" not in entity or "isasian" not in entity:
                    return False
                if entity["identity"].lower() not in valid_identities:
                    return False
                if entity["isasian"].lower() not in valid_isasian:
                    return False
            return True
        else:
            return False
    except json.JSONDecodeError:
        # 如果無法解析成 JSON，則返回 False
        return False

In [ ]:
import json
articles_dir = "../../test" #要正式跑的時候改成 ../../txt
results = []

# 遍歷所有文章文件
for filename in os.listdir(articles_dir):
    rerun_times = 3
    if filename.endswith(".txt"):  
        with open(os.path.join(articles_dir, filename), "r", encoding="utf-8") as file:
            text = file.read()
            content = classify_identities(text)
            content = content.split("<result>")[1].split("</result>")[0].lower()
            content_json = json.loads(content)
            if validate_json_and_identity(content_json):
                print(f"{filename} success")
            while not validate_json_and_identity(content_json) and rerun_times>0:
                content = classify_identities(text)
                content = content.split("<result>")[1].split("</result>")[0].lower()
                content_json = json.loads(content)
                rerun_times = rerun_times - 1
                if validate_json_and_identity(content_json):
                    print(f"{filename} success")
            if not validate_json_and_identity(content_json) and rerun_times == 0:
                print(f"{filename} failed")
            results.append(
                content_json
            )
print(results)


0.txt success
1.txt success
2.txt success
3.txt success
4.txt success
